# Read binary files generate by the modflow family

The format is the same for head files, cbc files and mt3dms conc files.

@TO 2021 10 30

Format from MODFLOW help site (under advanced, look for binary)

Binary files generated by MODFLOW are of several types.  These include:

    1	Head and Drawdown files (See CHEDFM and CDDNFM in Output Control.)
    2	Heads and flows interpolated to hydrogeologic units (See IOHUFHEADS and IOHUFFLOWS in the HUF2 package.)
    3.	Data related to subsidence or compaction (See the IBS, SUB, and SWT Packages.)
    4.	Cell-by-cell flow files (See ILPFCB in the LPF package and similar variables in other packages.)
    5.	Zeta values saved by the SWI2 package
    6.	Files saved by the SWR process. The formats for these files are not described here.

If you need to write code to read the binary files, the following descriptions of how the data is saved may help.
    hmtoggle_plus1	Array Data (#1 to #3)

The array data can be either in single-precision or double-precision format.  There is no data field that indicates which format is used.  Instead, you will need to try both and see which one works.

First read the following variables in order:

    KSTP: the time step number, an integer, 4 bytes.
    KPER: the stress period number, an integer, 4 bytes.
    PERTIM: the time in the current stress period, a real number, either 4 or 8 bytes.
    TOTIM, the total elapsed time, a real number, either 4 or 8 bytes.
    DESC, a description of the array, 16 ANSI characters, 16 bytes.
    NCOL, the number of columns in the array, an integer, 4 bytes.
    NROW, the number of rows in the array, an integer, 4 bytes.
    ILAY, the layer number, an integer, 4 bytes.
    
Next come a list of NROW x NCOL real numbers that represent the values of the array.  The values are in row major order.  Each value in the array occupies either 4 or 8 bytes depending on whether the values are in single- or double-precision.
    
After reading one set of values, start over with KSTP. Continue until reaching the end of the file.

The best way to read the binary file is using the regular python struct module.

In [2]:
import os
import struct
import numpy as np
import warnings
import pdb

In [3]:
def get_binary_fmt(fp):
    """Return format string suitable for struct module.
    
    We try both single and if it does not work doubple precision. The wrong one
    fails when converting the desc-bytes to a string.
    
    Parameters
    ----------
    fp: file pointer
        open binary file
    fmt: str
        the trial format string according to struct module with the parameters
        expected in the binary file and their byte lengths.
        
    Returns
    -------
    format: str
        complete and correct format string to be used with the struct module
    precision: str
        the single or double precision float format ('<i4' or '<i8')
    nrow: int
        number of rows
    ncol: int
        number of columns
    """
    fp.seek(0,0)
    try:
        fmt = '<iiff16siii'
        tpl = struct.unpack(fmt, fp.read(struct.calcsize(fmt)))
        kstp, kper, pertim, totim, desc, ncol, nrow, ilay = tpl # split the tuple
        desc = desc.decode('utf-8') # convert to str
        fp.seek(0, 0)
        #pdb.set_trace()
        return fmt + f'{ncol * nrow * 4:d}s', '<f4', ncol, nrow
    except:
        fmt = '<iidd16siii'
        tpl = struct.unpack(fmt, fp.read(struct.calcsize(fmt)))
        kstp, kper, pertim, totim, desc, ncol, nrow, ilay = tpl # split the tuple
        desc = desc.decode('utf-8') # convert to str
        fp.seek(0, 0)
        return fmt + f'{ncol * nrow * 8:d}s', '<f8', ncol, nrow
    else:
        raise ValueError("Can't get format for this presumed binary modflow file.")
        

def filelen(fp):
    """Return file length in bytes and rewind file."""
    L = fp.seek(0, 2)
    fp.seek(0, 0)
    return L

def recs_in_file(fp, fmt):
    """Return the total number of complete records in the file and the remainder bytes.
    
    Parameters
    ----------
    fp: file pointer
        open pointer to binary file
    fmt: str
        complete format according to struct (where array values are coded as a string,
        which will be docode later on to floats)
    """
    Lfile = filelen(fp)
    Lstr  = struct.calcsize(fmt)
    nrec = Lfile // Lstr # number of complete records
    nrem = Lfile % Lstr  # remainder in bytes of the file is incomplete or corrupt
    if nrem:
        warnings.warn(f'There is not a whole number of records in the file. nrec={nrec:d}, remainder={nrem:d} bytes.')
    return nrec, nrem

In [4]:
wd = '/Users/Theo/GRWMODELS/python/Nectaerra/Kruiszwin/cases/Kruiszwin_1/'
fname = 'justmodflow.hds'

# The binary file is read into a recarray

See the dtype for this array.

Only the complete records in the file are read. This also works with a corrupt or incomplete
binary file. So the data that can be read can still be used.


In [5]:
fp = open(os.path.join(wd, fname), 'rb')

fmt, precision, ncol, nrow = get_binary_fmt(fp) # get the correct format (by determining the file precision)

nrec, nrem = recs_in_file(fp, fmt) # get the number of complete records and see of there are any left-over bytes

print(f"The file's float precision = '{precision:s}'. Ncol={ncol:d}, Nrow={nrow:d}.") 
print(f"The number of complete rectords in the file is {nrec:d} with {nrem:d} remaining bytes.")
print("The file length is obtained from the file pointer and from the number of records\m")
print("with their length and left-over bytes. They should be the same.")
print("File lenght: {} and {}.".format(filelen(fp), nrec * struct.calcsize(fmt) + nrem))

# Define the dtype for the array to store all data per layer, kstp, kper as complete records
dtype = np.dtype([('kstp', '<i4'), ('kper', '<i4'),
                  ('pertim', precision), ('totim', precision),
                  ('desc', 'U', 16),
                  ('ncol', '<i4'), ('nrow', '<i4'), ('ilay', '<i4'), ('values', precision, (nrow, ncol))])

record = np.zeros(nrec, dtype=dtype) # Initialize the spaceto allow filling storing the data in a structured way.

buflen = struct.calcsize(fmt)

fp.seek(0, 0) # Not needed, but just to make sure

# Fill the container, record by record (and item by item for each record)
for i in range(nrec):
    buff = fp.read(buflen)

    kstp, kper, pertim, totim, desc, ncol, nrow, ilay, values = struct.unpack(fmt, buff)

    r = record[i]
    r['kstp'  ] = kstp
    r['kper'  ] = kper
    r['pertim'] = pertim
    r['totim' ] = totim
    r['desc'  ] = desc.decode('utf-8')
    r['ncol'  ] = ncol
    r['nrow'  ] = nrow
    r['ilay'  ] = ilay
    r['values'] = np.frombuffer(values, precision).reshape((nrow, ncol))  
fp.close()

# Print the last record
print("The last record:")
record[-1]

The file's float precision = '<f4'. Ncol=160, Nrow=1.
The number of complete rectords in the file is 443 with 92 remaining bytes.
The file length is obtained from the file pointer and from the number of records\m
with their length and left-over bytes. They should be the same.
File lenght: 303104 and 303104.
The last record:


/var/folders/90/m51x_b713y561gzh2kzy18d00000gq/T/ipykernel_51455/929645581.py:68: UserWarning: There is not a whole number of records in the file. nrec=443, remainder=92 bytes.
  warnings.warn(f'There is not a whole number of records in the file. nrec={nrec:d}, remainder={nrem:d} bytes.')


(1, 6, 180., 1080., '            HEAD', 160, 1, 73, [[-0.68      , -0.679999  , -0.67999804, -0.6799972 , -0.67999643, -0.6799958 , -0.6799951 , -0.6799946 , -0.67999405, -0.67999357, -0.6799932 , -0.67999285, -0.6799925 , -0.6799922 , -0.67999196, -0.6799918 , -0.67999154, -0.67999136, -0.67999125, -0.6799911 , -0.679991  , -0.6799909 , -0.67999077, -0.6799907 , -0.67999065, -0.6799906 , -0.67999053, -0.6799905 , -0.6799905 , -0.6799904 , -0.67999035, -0.67999035, -0.67999035, -0.6799903 , -0.6799903 , -0.6799903 , -0.67999023, -0.67999023, -0.67999023, -0.67999023, -0.67999023, -0.67999023, -0.67999023, -0.6799902 , -0.6799902 , -0.6799902 , -0.6799902 , -0.6799902 , -0.6799902 , -0.6799902 , -0.6799902 , -0.6799902 , -0.6799902 , -0.6799902 , -0.6799902 , -0.6799902 , -0.6799902 , -0.6799902 , -0.6799902 , -0.6799902 , -0.6799902 , -0.6799902 , -0.6799902 , -0.6799902 , -0.6799902 , -0.6799902 , -0.6799902 , -0.6799902 , -0.6799902 , -0.6799902 , -0.6799902 , -0.6799902 , -0.6799902

In [6]:
# With only one stress period we can construct the remaining array:

record['values'].shape

(443, 1, 160)

In [7]:
record.dtype

dtype([('kstp', '<i4'), ('kper', '<i4'), ('pertim', '<f4'), ('totim', '<f4'), ('desc', '<U16'), ('ncol', '<i4'), ('nrow', '<i4'), ('ilay', '<i4'), ('values', '<f4', (1, 160))])

In [11]:
record[:]['ilay']

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11,
       12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28,
       29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45,
       46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62,
       63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74,  1,  2,  3,  4,  5,
        6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22,
       23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39,
       40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56,
       57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73,
       74,  1,  2,  3,  4

In [12]:
record

array([(1, 1, 180.,  180., '            HEAD', 160, 1,  1, [[ 0.        ,  0.00757397,  0.0127422 ,  0.01578043,  0.0169417 ,  0.01645797,  0.01454128,  0.01138484,  0.0071648 ,  0.00204118, -0.0038408 , -0.01034974, -0.01736703, -0.02478597, -0.03251095, -0.04045669, -0.04854747, -0.05671648, -0.06490511, -0.07306234, -0.08114408, -0.08911262, -0.09693608, -0.10458779, -0.11204583, -0.11929259, -0.12631425, -0.13310035, -0.13964342, -0.14593866, -0.1519835 , -0.15777738, -0.16332142, -0.16861816, -0.17367138, -0.17848581, -0.183067  , -0.18742111, -0.19155484, -0.19547516, -0.19918932, -0.20270468, -0.20602873, -0.20916887, -0.21213244, -0.21492672, -0.21755886, -0.2200358 , -0.22236435, -0.22455111, -0.22660254, -0.2285249 , -0.2303242 , -0.23200637, -0.23357715, -0.235042  , -0.2364063 , -0.23767519, -0.23885368, -0.23982997, -0.24064675, -0.24145566, -0.24225529, -0.24304399, -0.24381983, -0.24458045, -0.24532297, -0.24604388, -0.24673884, -0.2474024 , -0.24802771, -0.24860601, -0.